# classifiers.ipynb

- It experiments with machine learning models ([Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html)) on different datasets.
- The goal is to evaluate the performance of these models with and without certain features (ergot-related predictors) to predict specific targets ("ergot_present_in_q3", "ergot_present_in_q4", "sum_severity_in_q3", "sum_severity_in_q4") in the datasets.



In [ ]:
from sklearn.ensemble import RandomForestClassifier  # type: ignore
from sklearn.svm import SVC, LinearSVC  # type: ignore
from evaluator import ModelEvaluator
import sys, warnings
import pandas as pd

sys.path.append("../")
from datasets.setCreator import SetCreator  # type: ignore
from datasets.setModifier import SetModifier  # type: ignore

Purpose :

- it defines lists of hyperparameters for the Random Forest Classifier and Support Vector Machine, filters out unnecessary warning messages, and prepares the datasets using a custom data set creator.

In [ ]:
N_ESTIMATORS = [100, 200, 500]
MAX_DEPTH = [5, 7, 10, 15]

C = [0.01, 0.25, 0.5, 0.75, 1]
GAMMA = [0.1, 0.5, 2, 5, 10]
DEGREE = [1, 2, 3]

In [ ]:
warnings.filterwarnings("ignore")

setModifier = SetModifier()
evaluator = ModelEvaluator()

setCreator = SetCreator()
dataset1 = setCreator.getSetList1()
dataset2 = setCreator.getSetList2()
dataset3 = setCreator.getSetList3()
dataset4 = setCreator.getSetList4()
dataset5 = setCreator.getSetList5()

# RandomForestClassifiers

Purpose:
- The purpose of this function is to test and evaluate the performance of a Random Forest Classifier on multiple datasets with a specified predictor variable, considering different combinations of hyperparameters (N_ESTIMATORS and MAX_DEPTH). 
- It aims to find the best model (in terms of the Area Under the ROC Curve - AUC) among the tested combinations.

Psuedocode:

- Loops through different datasets. 
- Test Random Forest Classifier model with different hyperparameters.
- Checks for the highest auc score.

Note :
- The difference between the two functions lies in how they handle the datasets: one function removes ergot-related predictors before training, while the other keeps them.
- A bunch of decision trees chosen at random and then merged together based on scoring.
- Prone to overfittin, hyperparameters are a must
- One of the most powerful algorithms

In [ ]:
def testWithErgotFeatures(predictor: str, datasets: list):
    print("Running the Random Forest Classifier:")
    print(f"Predicting: {predictor}\n\n")
    numExcludedSets = 0
    bestModel = {"auc": 0}

    numCombinations = len(datasets)

    for currData in datasets:
        trainningData = pd.DataFrame(currData["train"])
        trainY = trainningData[predictor]
        trainX = setModifier.rmErgotPredictors(trainningData)

        testData = pd.DataFrame(currData["test"])
        testY = testData[predictor]
        testX = setModifier.rmErgotPredictors(testData)

        for estimator in N_ESTIMATORS:
            for depth in MAX_DEPTH:
                try:
                    rfc = RandomForestClassifier(
                        random_state=5, n_estimators=estimator, max_depth=depth
                    )
                    rfc.fit(trainX, trainY)
                    results = evaluator.evaluateClassification(
                        rfc, currData["desc"], trainX, trainY, testX, testY
                    )

                    if results["auc"] > bestModel["auc"]:
                        bestModel = results

                        bestModel["n_estimators"] = estimator
                        bestModel["max_depth"] = depth

                except Exception as e:
                    numExcludedSets += 1

    print(
        f"[{numCombinations - numExcludedSets}/{numCombinations}] sets were tested on the Random Forest Classifier"
    )
    print(f"The best model, as per roc, was:\n {bestModel}")

In [ ]:
def testWithoutErgotFeatures(predictor: str, datasets: list):
    print("Running the Random Forest Classifier:")
    print(f"Predicting: {predictor}\n\n")
    numExcludedSets = 0
    bestModel = {"auc": 0}

    numCombinations = len(datasets)

    for currData in datasets:
        trainningData = pd.DataFrame(currData["train"])
        trainY = trainningData[predictor]
        trainX = setModifier.rmErgotFeatures(trainningData)

        testData = pd.DataFrame(currData["test"])
        testY = testData[predictor]
        testX = setModifier.rmErgotFeatures(testData)

        for estimator in N_ESTIMATORS:
            for depth in MAX_DEPTH:
                try:
                    rfc = RandomForestClassifier(
                        random_state=5, n_estimators=estimator, max_depth=depth
                    )
                    rfc.fit(trainX, trainY)
                    results = evaluator.evaluateClassification(
                        rfc, currData["desc"], trainX, trainY, testX, testY
                    )

                    if results["auc"] > bestModel["auc"]:
                        bestModel = results

                        bestModel["n_estimators"] = estimator
                        bestModel["max_depth"] = depth

                except Exception as e:
                    numExcludedSets += 1

    print(
        f"[{numCombinations - numExcludedSets}/{numCombinations}] sets were tested on the Random Forest Classifier"
    )
    print(f"The best model, as per roc, was:\n {bestModel}")

In [ ]:
testWithErgotFeatures("ergot_present_in_q3", dataset1)
testWithoutErgotFeatures("ergot_present_in_q3", dataset1)
testWithErgotFeatures("ergot_present_in_q3", dataset2)
testWithoutErgotFeatures("ergot_present_in_q3", dataset2)
testWithErgotFeatures("ergot_present_in_q3", dataset3)
testWithoutErgotFeatures("ergot_present_in_q3", dataset3)
testWithErgotFeatures("ergot_present_in_q3", dataset4)
testWithoutErgotFeatures("ergot_present_in_q3", dataset4)
testWithErgotFeatures("ergot_present_in_q3", dataset5)
testWithoutErgotFeatures("ergot_present_in_q3", dataset5)

In [ ]:
testWithErgotFeatures("ergot_present_in_q4", dataset1)
testWithoutErgotFeatures("ergot_present_in_q4", dataset1)
testWithErgotFeatures("ergot_present_in_q4", dataset2)
testWithoutErgotFeatures("ergot_present_in_q4", dataset2)
testWithErgotFeatures("ergot_present_in_q4", dataset3)
testWithoutErgotFeatures("ergot_present_in_q4", dataset3)
testWithErgotFeatures("ergot_present_in_q4", dataset4)
testWithoutErgotFeatures("ergot_present_in_q4", dataset4)
testWithErgotFeatures("ergot_present_in_q4", dataset5)
testWithoutErgotFeatures("ergot_present_in_q4", dataset5)

In [ ]:
testWithErgotFeatures("sum_severity_in_q3", dataset1)
testWithoutErgotFeatures("sum_severity_in_q3", dataset1)
testWithErgotFeatures("sum_severity_in_q3", dataset2)
testWithoutErgotFeatures("sum_severity_in_q3", dataset2)
testWithErgotFeatures("sum_severity_in_q3", dataset3)
testWithoutErgotFeatures("sum_severity_in_q3", dataset3)
testWithErgotFeatures("sum_severity_in_q3", dataset4)
testWithoutErgotFeatures("sum_severity_in_q3", dataset4)
testWithErgotFeatures("sum_severity_in_q3", dataset5)
testWithoutErgotFeatures("sum_severity_in_q3", dataset5)

In [ ]:
testWithErgotFeatures("sum_severity_in_q4", dataset1)
testWithoutErgotFeatures("sum_severity_in_q4", dataset1)
testWithErgotFeatures("sum_severity_in_q4", dataset2)
testWithoutErgotFeatures("sum_severity_in_q4", dataset2)
testWithErgotFeatures("sum_severity_in_q4", dataset3)
testWithoutErgotFeatures("sum_severity_in_q4", dataset3)
testWithErgotFeatures("sum_severity_in_q4", dataset4)
testWithoutErgotFeatures("sum_severity_in_q4", dataset4)
testWithErgotFeatures("sum_severity_in_q4", dataset5)
testWithoutErgotFeatures("sum_severity_in_q4", dataset5)

# Support Vector Machine

Purpose :
- The purpose of this code is to test and find the best configuration for a Linear Support Vector Machine (SVM) classifier with different values of the regularization parameter 'C' on various datasets.
- It aims to find the best model (in terms of the Area Under the ROC Curve - AUC) among the tested combinations.

Psuedocode : 
- Loops through different datasets. 
- Train Support Vector Machine model with different hyperparameters.
- Checks for the highest auc score.

Note :
- SVM better for smaller datasets, vulnerable to noise - therefore using feature reduction first
- works even worse if lots of zero values
- Handles outliers great - soft margines

In [ ]:
def SVMLinearWithErgotFeatures(predictor: str, datasets: list):
    print("Running linear SVM Classifier:")
    print(f"Predicting: {predictor}\n\n")
    numExcludedSets = 0
    bestModel = {"auc": 0.0}

    numCombinations = len(datasets)

    for currData in datasets:
        trainningData = currData["train"]
        trainY = trainningData[predictor]
        trainX = setModifier.rmErgotPredictors(trainningData)

        testData = currData["test"]
        testY = testData[predictor]
        testX = setModifier.rmErgotPredictors(testData)

        for c in C:
            try:
                smv = LinearSVC(C=c, random_state=0)
                smv.fit(trainX, trainY)
                results = evaluator.evaluateClassification(
                    smv,
                    currData["desc"],
                    trainX,
                    trainY,
                    testX,
                    testY,
                    hasFeatImportance=False,
                )

                if results["auc"] > bestModel["auc"]:
                    bestModel = results

                    bestModel["c"] = c

            except Exception as e:
                numExcludedSets += 1

    print(
        f"[{numCombinations - numExcludedSets}/{numCombinations}] sets were tested on the linear SVM Classifier"
    )
    print(f"The best model, as per roc, was:\n {bestModel}")

Purpose :
- The purpose of this code is to test and find the best configuration for a Support Vector Machine (SVM) classifier with a polynomial kernel on various datasets. It iterates through a list of datasets.
- It aims to find the best model (in terms of the Area Under the ROC Curve - AUC) among the tested combinations.

Psuedocode : 
- Loops through different datasets. 
- Trains the classifier using different values of the regularization parameter 'C' and the polynomial degree 'deg', evaluates the performance using the AUC (Area Under the Receiver Operating Characteristic Curve) metric.
- Checks for the highest auc score.

In [ ]:
def SVMPolyWithErgotFeatures(predictor: str, datasets: list):
    print("Running poly kernal SVM Classifier:")
    print(f"Predicting: {predictor}\n\n")
    numExcludedSets = 0
    bestModel = {"auc": 0.0}

    numCombinations = len(datasets)

    for currData in datasets:
        trainningData = currData["train"]
        trainY = trainningData[predictor]
        trainX = setModifier.rmErgotPredictors(trainningData)

        testData = currData["test"]
        testY = testData[predictor]
        testX = setModifier.rmErgotPredictors(testData)

        for c in C:
            for deg in DEGREE:
                try:
                    smv = SVC(kernel="poly", degree=deg, coef0=1, C=c, random_state=0)
                    smv.fit(trainX, trainY)
                    results = evaluator.evaluateClassification(
                        smv,
                        currData["desc"],
                        trainX,
                        trainY,
                        testX,
                        testY,
                        hasFeatImportance=False,
                    )

                    if results["auc"] > bestModel["auc"]:
                        bestModel = results

                        bestModel["c"] = c
                        bestModel["degree"] = deg

                except Exception as e:
                    numExcludedSets += 1

    print(
        f"[{numCombinations - numExcludedSets}/{numCombinations}] sets were tested on the poly kernal SVM Classifier"
    )
    print(f"The best model, as per roc, was:\n {bestModel}")

Purpose :
- The purpose of this code is to test and find the best configuration for a Support Vector Machine (SVM) classifier with a radial basis function (RBF) kernel on various datasets.
- It aims to find the best model (in terms of the Area Under the ROC Curve - AUC) among the tested combinations.

Psuedocode : 
- Loops through different datasets. 
- Trains the classifier using different values of the regularization parameter 'C' and the kernel coefficient 'gamma', evaluates the performance using the AUC (Area Under the Receiver Operating Characteristic Curve) metric.
- Checks for the highest auc score.

In [ ]:
def SVMRBFWithErgotFeatures(predictor: str, datasets: list):
    print("Running RBF SVM Classifier:")
    print(f"Predicting: {predictor}\n\n")
    numExcludedSets = 0
    bestModel = {"auc": 0.0}

    numCombinations = len(datasets)

    for currData in datasets:
        trainningData = currData["train"]
        trainY = trainningData[predictor]
        trainX = setModifier.rmErgotPredictors(trainningData)

        testData = currData["test"]
        testY = testData[predictor]
        testX = setModifier.rmErgotPredictors(testData)

        for c in C:
            for gam in GAMMA:
                try:
                    smv = SVC(kernel="rbf", gamma=gam, C=c, random_state=0)
                    smv.fit(trainX, trainY)
                    results = evaluator.evaluateClassification(
                        smv,
                        currData["desc"],
                        trainX,
                        trainY,
                        testX,
                        testY,
                        hasFeatImportance=False,
                    )

                    if results["auc"] > bestModel["auc"]:
                        bestModel = results

                        bestModel["c"] = c
                        bestModel["gamma"] = gam

                except Exception as e:
                    numExcludedSets += 1

    print(
        f"[{numCombinations - numExcludedSets}/{numCombinations}] sets were tested on the RBF SVM Classifier"
    )
    print(f"The best model, as per roc, was:\n {bestModel}")

In [ ]:
SVMLinearWithErgotFeatures("ergot_present_in_q3", dataset2)
SVMPolyWithErgotFeatures("ergot_present_in_q3", dataset2)
SVMRBFWithErgotFeatures("ergot_present_in_q3", dataset2)
SVMLinearWithErgotFeatures("ergot_present_in_q3", dataset3)
SVMPolyWithErgotFeatures("ergot_present_in_q3", dataset3)
SVMRBFWithErgotFeatures("ergot_present_in_q3", dataset3)
SVMLinearWithErgotFeatures("ergot_present_in_q3", dataset4)
SVMPolyWithErgotFeatures("ergot_present_in_q3", dataset4)
SVMRBFWithErgotFeatures("ergot_present_in_q3", dataset4)
SVMLinearWithErgotFeatures("ergot_present_in_q3", dataset5)
SVMPolyWithErgotFeatures("ergot_present_in_q3", dataset5)
SVMRBFWithErgotFeatures("ergot_present_in_q3", dataset5)

In [ ]:
SVMLinearWithErgotFeatures("ergot_present_in_q4", dataset2)
SVMPolyWithErgotFeatures("ergot_present_in_q4", dataset2)
SVMRBFWithErgotFeatures("ergot_present_in_q4", dataset2)
SVMLinearWithErgotFeatures("ergot_present_in_q4", dataset3)
SVMPolyWithErgotFeatures("ergot_present_in_q4", dataset3)
SVMRBFWithErgotFeatures("ergot_present_in_q4", dataset3)
SVMLinearWithErgotFeatures("ergot_present_in_q4", dataset4)
SVMPolyWithErgotFeatures("ergot_present_in_q4", dataset4)
SVMRBFWithErgotFeatures("ergot_present_in_q4", dataset4)
SVMLinearWithErgotFeatures("ergot_present_in_q4", dataset5)
SVMPolyWithErgotFeatures("ergot_present_in_q4", dataset5)
SVMRBFWithErgotFeatures("ergot_present_in_q4", dataset5)

In [ ]:
SVMLinearWithErgotFeatures("sum_severity_in_q3", dataset2)
SVMPolyWithErgotFeatures("sum_severity_in_q3", dataset2)
SVMRBFWithErgotFeatures("sum_severity_in_q3", dataset2)
SVMLinearWithErgotFeatures("sum_severity_in_q3", dataset3)
SVMPolyWithErgotFeatures("sum_severity_in_q3", dataset3)
SVMRBFWithErgotFeatures("sum_severity_in_q3", dataset3)
SVMLinearWithErgotFeatures("sum_severity_in_q3", dataset4)
SVMPolyWithErgotFeatures("sum_severity_in_q3", dataset4)
SVMRBFWithErgotFeatures("sum_severity_in_q3", dataset4)
SVMLinearWithErgotFeatures("sum_severity_in_q3", dataset5)
SVMPolyWithErgotFeatures("sum_severity_in_q3", dataset5)
SVMRBFWithErgotFeatures("sum_severity_in_q3", dataset5)

In [ ]:
SVMLinearWithErgotFeatures("sum_severity_in_q4", dataset2)
SVMPolyWithErgotFeatures("sum_severity_in_q4", dataset2)
SVMRBFWithErgotFeatures("sum_severity_in_q4", dataset2)
SVMLinearWithErgotFeatures("sum_severity_in_q4", dataset3)
SVMPolyWithErgotFeatures("sum_severity_in_q4", dataset3)
SVMRBFWithErgotFeatures("sum_severity_in_q4", dataset3)
SVMLinearWithErgotFeatures("sum_severity_in_q4", dataset4)
SVMPolyWithErgotFeatures("sum_severity_in_q4", dataset4)
SVMRBFWithErgotFeatures("sum_severity_in_q4", dataset4)
SVMLinearWithErgotFeatures("sum_severity_in_q4", dataset5)
SVMPolyWithErgotFeatures("sum_severity_in_q4", dataset5)
SVMRBFWithErgotFeatures("sum_severity_in_q4", dataset5)